# NFCS Demo: Complex Ginzburg-Landau Solver

This notebook demonstrates the core functionality of the Neural Field Control System, focusing on the Complex Ginzburg-Landau (CGL) equation solver.

## Mathematical Background

The CGL equation describes the evolution of the complex neural field φ(x,t):

$$\frac{\partial\phi}{\partial t} = \phi + (1 + ic_1)\nabla^2\phi - (1 + ic_3)|\phi|^2\phi + u(x,t)$$

Where:
- φ(x,t) is the complex neural field
- c₁ controls linear dispersion
- c₃ controls nonlinear self-action
- u(x,t) is external control

## Key Features

1. **Split-step Fourier method** for numerical integration
2. **Topological defect tracking** in the phase field
3. **Benjamin-Feir instability** demonstration
4. **Energy conservation** analysis

In [ ]:
# Setup and imports
import sys
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Add src to path
sys.path.insert(0, str(Path.cwd().parent / "src"))

from core.cgl_solver import CGLSolver
from core.state import CGLConfig
from utils.config_loader import load_config

# Setup plotting
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

print("🧠 NFCS CGL Solver Demo")
print("Modules loaded successfully!")

## 1. Basic Solver Setup

In [ ]:
# Create configuration
config = CGLConfig(
    c1=0.5,              # Linear dispersion
    c3=1.0,              # Nonlinear coefficient  
    grid_size=(128, 128), # 2D grid
    time_step=0.01,       # Time step
    spatial_extent=(8.0, 8.0),  # Domain size
    boundary_conditions="periodic"
)

# Initialize solver
solver = CGLSolver(config)

# Display grid information
grid_info = solver.get_grid_info()
print("Grid Information:")
for key, value in grid_info.items():
    print(f"  {key}: {value}")

## 2. Initial Conditions and Patterns

In [ ]:
# Create different initial patterns
patterns = {
    'Plane Wave': solver.create_initial_condition(
        pattern="plane_wave", amplitude=0.5, kx=1.0, ky=0.5
    ),
    'Gaussian Blob': solver.create_initial_condition(
        pattern="gaussian", amplitude=0.8, sigma=2.0
    ),
    'Spiral (m=1)': solver.create_initial_condition(
        pattern="spiral", amplitude=0.6, m=1, k_radial=0.3
    ),
    'Random Noise': solver.create_initial_condition(
        pattern="random_noise", amplitude=0.2
    )
}

# Visualize initial patterns
fig, axes = plt.subplots(2, 4, figsize=(16, 8))

for i, (name, field) in enumerate(patterns.items()):
    # Amplitude
    im1 = axes[0, i].imshow(np.abs(field), cmap='viridis', origin='lower')
    axes[0, i].set_title(f'{name}\nAmplitude |φ|')
    plt.colorbar(im1, ax=axes[0, i])
    
    # Phase
    im2 = axes[1, i].imshow(np.angle(field), cmap='hsv', origin='lower', vmin=-np.pi, vmax=np.pi)
    axes[1, i].set_title(f'Phase arg(φ)')
    plt.colorbar(im2, ax=axes[1, i])

plt.tight_layout()
plt.show()

## 3. Time Evolution Without Control

In [ ]:
# Choose initial condition
initial_field = patterns['Spiral (m=1)'].copy()
control_field = np.zeros_like(initial_field)  # No external control

# Evolution parameters
n_steps = 200
save_every = 40  # Save snapshots every 40 steps

# Storage for evolution
field = initial_field.copy()
snapshots = [field.copy()]
times = [0.0]
energies = [solver.compute_energy(field)]

# Time evolution loop
print(f"Evolving for {n_steps} steps...")
for step in range(n_steps):
    field = solver.step(field, control_field)
    
    if step % save_every == 0 or step == n_steps - 1:
        snapshots.append(field.copy())
        times.append((step + 1) * config.time_step)
        energies.append(solver.compute_energy(field))
        
        if step % (n_steps // 5) == 0:
            print(f"  Step {step+1:3d}: Energy = {energies[-1]:.4f}")

print("Evolution complete!")
print(f"Final energy: {energies[-1]:.4f}")
print(f"Energy change: {(energies[-1] - energies[0])/energies[0]*100:.2f}%")

In [ ]:
# Visualize evolution snapshots
n_snapshots = len(snapshots)
cols = min(6, n_snapshots)
rows = 2

fig, axes = plt.subplots(rows, cols, figsize=(3*cols, 6))
if cols == 1:
    axes = axes.reshape(-1, 1)

for i in range(cols):
    if i < n_snapshots:
        snapshot = snapshots[i]
        time = times[i]
        
        # Amplitude
        im1 = axes[0, i].imshow(np.abs(snapshot), cmap='viridis', origin='lower')
        axes[0, i].set_title(f't = {time:.2f}\n|φ|')
        
        # Phase
        im2 = axes[1, i].imshow(np.angle(snapshot), cmap='hsv', origin='lower', 
                                vmin=-np.pi, vmax=np.pi)
        axes[1, i].set_title(f'arg(φ)')
        
        # Remove ticks for clarity
        axes[0, i].set_xticks([])
        axes[0, i].set_yticks([])
        axes[1, i].set_xticks([])
        axes[1, i].set_yticks([])

plt.tight_layout()
plt.show()

In [ ]:
# Energy evolution plot
plt.figure(figsize=(10, 6))

plt.subplot(1, 2, 1)
plt.plot(times, energies, 'b-', linewidth=2, marker='o')
plt.xlabel('Time')
plt.ylabel('Energy')
plt.title('Energy Evolution')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
energy_change = [(e - energies[0])/energies[0]*100 for e in energies]
plt.plot(times, energy_change, 'r-', linewidth=2, marker='s')
plt.xlabel('Time')
plt.ylabel('Energy Change (%)')
plt.title('Relative Energy Change')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Benjamin-Feir Instability

When c₁ · c₃ > 0, plane waves become unstable and develop modulation, potentially leading to defect formation.

In [ ]:
# Configure for Benjamin-Feir instability
bf_config = CGLConfig(
    c1=1.0,  # Positive
    c3=1.0,  # Positive -> c1*c3 = 1 > 0 (unstable)
    grid_size=(128, 128),
    time_step=0.005,  # Smaller step for stability
    spatial_extent=(10.0, 10.0)
)

bf_solver = CGLSolver(bf_config)

print(f"Benjamin-Feir condition: c1*c3 = {bf_config.c1 * bf_config.c3} > 0")
print("This should lead to modulation instability!")

# Start with plane wave + small perturbation
plane_wave = bf_solver.create_initial_condition(
    pattern="plane_wave", amplitude=0.8, kx=0.8, ky=0.0
)
perturbation = bf_solver.create_initial_condition(
    pattern="random_noise", amplitude=0.02
)
bf_field = plane_wave + perturbation

# Evolve and track amplitude variation
n_steps_bf = 300
bf_snapshots = [bf_field.copy()]
bf_times = [0.0]
amplitude_variations = []

print("\nEvolution with Benjamin-Feir instability:")
for step in range(n_steps_bf):
    bf_field = bf_solver.step(bf_field, np.zeros_like(bf_field))
    
    if step % 50 == 0 or step == n_steps_bf - 1:
        bf_snapshots.append(bf_field.copy())
        bf_times.append((step + 1) * bf_config.time_step)
        
        # Measure amplitude variation
        amplitude = np.abs(bf_field)
        amp_var = (np.max(amplitude) - np.min(amplitude)) / np.max(amplitude)
        amplitude_variations.append(amp_var)
        
        print(f"  Step {step+1:3d}: Amplitude variation = {amp_var:.4f}")

print(f"\nFinal amplitude variation: {amplitude_variations[-1]:.4f}")
if amplitude_variations[-1] > 0.3:
    print("✅ Significant modulation developed - Benjamin-Feir instability observed!")
else:
    print("⚠️  Limited modulation - may need longer evolution time")

In [ ]:
# Visualize Benjamin-Feir evolution
n_bf_snapshots = len(bf_snapshots)
cols = min(6, n_bf_snapshots)

fig, axes = plt.subplots(2, cols, figsize=(3*cols, 6))

for i in range(cols):
    if i < n_bf_snapshots:
        snapshot = bf_snapshots[i]
        time = bf_times[i] if i < len(bf_times) else 0
        
        # Amplitude (focus on modulation)
        amplitude = np.abs(snapshot)
        im1 = axes[0, i].imshow(amplitude, cmap='viridis', origin='lower')
        axes[0, i].set_title(f't = {time:.3f}\n|φ|')
        
        # Amplitude cross-section to show modulation
        mid_y = amplitude.shape[0] // 2
        x_profile = amplitude[mid_y, :]
        axes[1, i].plot(x_profile, 'b-', linewidth=2)
        axes[1, i].set_title('Amplitude Profile')
        axes[1, i].set_xlabel('X')
        axes[1, i].set_ylabel('|φ|')
        axes[1, i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Plot amplitude variation over time
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
valid_times = bf_times[1:len(amplitude_variations)+1] if len(amplitude_variations) < len(bf_times) else bf_times[:len(amplitude_variations)]
plt.plot(valid_times, amplitude_variations, 'r-', linewidth=2, marker='o')
plt.xlabel('Time')
plt.ylabel('Amplitude Variation')
plt.title('Benjamin-Feir Modulation Growth')
plt.grid(True, alpha=0.3)

# Final amplitude map
plt.subplot(1, 2, 2)
final_amplitude = np.abs(bf_snapshots[-1])
im = plt.imshow(final_amplitude, cmap='viridis', origin='lower')
plt.title('Final Amplitude Distribution')
plt.colorbar(im)

plt.tight_layout()
plt.show()

## 5. Control Field Effects

Demonstrate how external control u(x,t) can influence the field evolution.

In [ ]:
# Reset to stable configuration
control_config = CGLConfig(
    c1=0.5,
    c3=1.0,
    grid_size=(64, 64),
    time_step=0.01,
    spatial_extent=(6.0, 6.0)
)

control_solver = CGLSolver(control_config)

# Initial condition
initial = control_solver.create_initial_condition(
    pattern="gaussian", amplitude=0.5, sigma=1.5
)

# Create spatially localized control
nx, ny = control_config.grid_size
x = np.linspace(-3, 3, nx)
y = np.linspace(-3, 3, ny)
X, Y = np.meshgrid(x, y, indexing='ij')

# Gaussian control pulse (off-center)
control_amplitude = 0.3
control_center_x = 1.0
control_center_y = 0.0
control_width = 0.8

control_field = control_amplitude * np.exp(
    -((X - control_center_x)**2 + (Y - control_center_y)**2) / (2 * control_width**2)
).astype(np.complex128)

print(f"Control field: Gaussian pulse at ({control_center_x}, {control_center_y})")
print(f"Control amplitude: {control_amplitude}")
print(f"Control width: {control_width}")

In [ ]:
# Compare evolution with and without control
n_steps_control = 100

# Without control
field_no_control = initial.copy()
snapshots_no_control = [field_no_control.copy()]

# With control
field_with_control = initial.copy()
snapshots_with_control = [field_with_control.copy()]

print("Comparing evolution with and without control...")
for step in range(n_steps_control):
    # No control case
    field_no_control = control_solver.step(
        field_no_control, 
        np.zeros_like(field_no_control)
    )
    
    # With control case
    field_with_control = control_solver.step(
        field_with_control, 
        control_field
    )
    
    # Save snapshots
    if step % 20 == 0 or step == n_steps_control - 1:
        snapshots_no_control.append(field_no_control.copy())
        snapshots_with_control.append(field_with_control.copy())

print("Evolution complete!")

In [ ]:
# Visualize control effects
n_control_snapshots = len(snapshots_no_control)
cols = min(5, n_control_snapshots)

fig, axes = plt.subplots(3, cols, figsize=(3*cols, 9))

for i in range(cols):
    if i < n_control_snapshots:
        time = i * 20 * control_config.time_step
        
        # Without control
        field1 = snapshots_no_control[i]
        im1 = axes[0, i].imshow(np.abs(field1), cmap='viridis', origin='lower')
        axes[0, i].set_title(f't = {time:.2f}\nNo Control')
        
        # With control
        field2 = snapshots_with_control[i]
        im2 = axes[1, i].imshow(np.abs(field2), cmap='viridis', origin='lower')
        axes[1, i].set_title(f'With Control')
        
        # Difference
        diff = np.abs(field2) - np.abs(field1)
        im3 = axes[2, i].imshow(diff, cmap='RdBu', origin='lower')
        axes[2, i].set_title(f'Difference')
        
        # Remove ticks
        for row in range(3):
            axes[row, i].set_xticks([])
            axes[row, i].set_yticks([])

plt.tight_layout()
plt.show()

# Show control field
plt.figure(figsize=(10, 4))

plt.subplot(1, 3, 1)
plt.imshow(np.abs(control_field), cmap='Reds', origin='lower')
plt.title('Control Field |u|')
plt.colorbar()

plt.subplot(1, 3, 2)
plt.imshow(np.abs(snapshots_no_control[-1]), cmap='viridis', origin='lower')
plt.title('Final: No Control')
plt.colorbar()

plt.subplot(1, 3, 3)
plt.imshow(np.abs(snapshots_with_control[-1]), cmap='viridis', origin='lower')
plt.title('Final: With Control')
plt.colorbar()

plt.tight_layout()
plt.show()

## 6. Summary and Key Insights

This demonstration has shown:

1. **Split-step method**: Accurate numerical integration of the CGL equation
2. **Pattern formation**: Various initial conditions lead to different dynamics
3. **Benjamin-Feir instability**: Parameter regimes where plane waves become unstable
4. **Control effects**: External fields can significantly influence evolution
5. **Energy dynamics**: Energy conservation and dissipation in different regimes

The CGL solver forms the foundation of the NFCS neural field dynamics, providing the continuous substrate on which cognitive processes operate.

In [ ]:
# Final summary statistics
print("🧠 CGL Solver Demo Summary")
print("=" * 40)
print(f"✅ Tested {len(patterns)} initial condition types")
print(f"✅ Demonstrated Benjamin-Feir instability (c1*c3 > 0)")
print(f"✅ Showed control field effects")
print(f"✅ Verified energy evolution tracking")
print("\n🎯 The CGL solver is ready for integration into the full NFCS system!")

# Performance info
total_steps = n_steps + n_steps_bf + 2 * n_steps_control
print(f"\n📊 Performance: {total_steps} total integration steps completed")
print(f"   Grid resolution: {config.grid_size}")
print(f"   Time step: {config.time_step}")